In [ ]:
import os

class TextWebUI:
    class Config:
        def __init__(self):
            self.use_google_drive = False #@param {type:"boolean"}
            self.share_gradio = True #@param {type:"boolean"}
            self.commit = '3c076c3' #@param {type:"string"}
            self.username = 'a' #@param {type:"string"}
            self.password = 'a' #@param {type:"string"}
            self.ngrok_token = '' #@param {type: "string"}
            self.hf_user = '' #@param {type: "string"}
            self.hf_pass = '' #@param {type: "string"}
            self.models_to_download = '' #@param {type: "string"}
            self.repo_dir = 'text-generation-webui'
            self.commit_file = f'{self.repo_dir}/commit.txt'
            self.workdir = '/content'

    class Args:
        def __init__(self, setup):
            self.setup = setup
            self.config = setup.config
            self.logger = setup.logger

        def args(self):
            args = []
            args.append(self.ngrok_args())
            args.append(self.share_args())
            args.append(self.gradio_auth_args())
            return ' '.join(args)

        def environment_variables(self):
            if self.config.hf_user == '' or self.config.hf_pass == '':
                return ''
            else:
                return f'HF_USER={self.config.hf_user} HF_PASS={self.config.hf_pass}'

        def ngrok_args(self):
            if self.config.ngrok_token != '':
                self.log('Using ngrok')
                return f'--ngrok_token {self.config.ngrok_token}'
            else:
                return ''
        
        def share_args(self):
            if self.config.share_gradio and self.config.ngrok_token == '':
                self.log('Sharing on Gradio')
                return '--share'
            else:
                self.log('Not sharing on Gradio')
                return ''
            
        def gradio_auth_args(self):
            if self.config.username != '' and self.config.password != '':
                self.log('Using Gradio auth')
                return f'--gradio-auth {self.config.username}:{self.config.password}'
            else:
                self.log('Not using Gradio auth')
                return ''
            
        def log(self, message):
            self.logger.log(f'Args - {message}')     
        
    class Logger:
        def __init__(self):
            pass

        def log(self, message):
            print(message)
        
    class Setup:
        def __init__(self):
            self.logger = TextWebUI.Logger()
            self.config = TextWebUI.Config()
            self.args = TextWebUI.Args(self)

        def setup(self):
            self.log('Setting up...')
            self.init_google_drive()
            %cd {self.config.workdir}
            self.clone_repo_if_needed()
            %cd {self.config.repo_dir}
            self.log(f'Directory: {os.getcwd()}')
            self.log('Loading requirements...')
            %pip install -q -r requirements.txt
            self.log('Loaded')
            self.preload_models()

        def download_webui(self):
            self.log(f'Downloading webui commit {self.config.commit} into {self.config.repo_dir}...')
            !curl https://github.com/oobabooga/text-generation-webui/archive/{self.config.commit}.zip -L -s -o text-generation-webui.zip
            !unzip -q text-generation-webui.zip
            !rm -rf {self.config.repo_dir}
            !mv text-generation-webui-* {self.config.repo_dir}
            !echo {self.config.commit} > {self.config.commit_file}
            !rm text-generation-webui.zip
            self.log('Downloaded')

        def init_google_drive(self):
            if self.config.use_google_drive:
                self.log('Mounting Google Drive...')
                from google.colab import drive
                drive.mount(f'{self.config.workdir}/drive')
                self.config.workdir = f'{self.config.workdir}/drive/MyDrive'
                self.log(f'Mounted on: {self.config.workdir}')

        def clone_repo_if_needed(self):
            if os.path.isdir(self.config.repo_dir):
                old_commit = open(self.config.commit_file, 'r').read()
                if old_commit != self.config.commit:
                    self.download_webui()
                else:
                    self.log('Repo already up to date')
            else:
                self.download_webui()   

        def preload_models(self):
            if self.config.models_to_download != '':
                self.log('Preloading models...')
                threads = os.cpu_count() - 1
                threads = 1 if threads < 1 else threads
                self.log(f'Using {threads} threads')
                for model in self.config.models_to_download.split(','):
                    self.log(f'Downloading model {model} ...')
                    !{self.args.environment_variables()} python download-model.py {model} --threads {threads}

        def init_ngrok(self):
            if self.config.ngrok_token != '':
                self.log('Installing ngrok...')
                %pip install pyngrok

        def log(self, message):
            self.logger.log(f'Setup - {message}')        

    @classmethod
    def start_ui(cls):
        setup = cls.Setup()
        args = setup.args
        setup.setup()
        setup.logger.log('Starting server...')
        !{args.environment_variables()} python server.py {args.args()}

TextWebUI.start_ui()